In [1]:
!pip install pytorch-optimizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 7.5 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import math

def create_1d_rope_embeddings(seq_len, dim, device='cpu'):
    # Ensure the dimension is even
    assert dim % 2 == 0, "Dimension must be even for 1D RoPE"

    # Create position indices
    inv_freq = 1.0 / (10000 ** (torch.arange(0, dim, 2).float() / dim))
    inv_freq = inv_freq.to(device)

    # Create position embeddings
    sinusoid_inp = torch.einsum("i,j->ij", torch.arange(seq_len, dtype=torch.float, device=device), inv_freq)

    # Create the RoPE embeddings
    rope_sin = torch.sin(sinusoid_inp).unsqueeze(1).permute(1, 0, 2)
    rope_cos = torch.cos(sinusoid_inp).unsqueeze(1).permute(1, 0, 2)

    return rope_sin, rope_cos

def apply_1d_rope(token_embeddings, rope_sin, rope_cos):
    # Ensure the token embeddings dimension is even
    assert token_embeddings.shape[-1] % 2 == 0, "Token embeddings dimension must be even for 1D RoPE"

    # Split the token embeddings into even and odd dimensions
    token_embeddings_even = token_embeddings[..., 0::2]
    token_embeddings_odd = token_embeddings[..., 1::2]

    # Apply the RoPE embeddings
    token_embeddings_even_rotated = token_embeddings_even * rope_cos - token_embeddings_odd * rope_sin
    token_embeddings_odd_rotated = token_embeddings_even * rope_sin + token_embeddings_odd * rope_cos

    # Concatenate the rotated embeddings
    token_embeddings_rotated = torch.cat((token_embeddings_even_rotated, token_embeddings_odd_rotated), dim=-1)
    return token_embeddings_rotated

In [3]:
# For demonstration, assume we have a list of texts
texts = [
    "forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six",
    "twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six",
    "forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five",
    "thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two",
    "thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four",
    "forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one",
    "thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight",
    "five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four",
    "fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine",
    "ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine",
    "fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine one hundred",
    "seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six",
    "fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine one hundred",
    "twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six",
    "thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six",
    "twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four",
    "thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two",
    "twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six",
    "thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five",
    "thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two",
    "six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five",
    "twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one",
    "thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven",
    "forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three",
    "thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two",
    "seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six",
    "forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four",
    "forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four",
    "twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three",
    "fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three",
    "fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three",
    "thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three",
    "twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six",
    "three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two",
    "twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five",
    "twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one",
    "twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy",
    "twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight",
    "seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six",
    "thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two",
    "fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine one hundred",
    "thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight",
    "sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five",
    "twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one",
    "forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three",
    "fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four",
    "twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy",
    "twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven",
    "thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two",
    "three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two",
    "thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five",
    "two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one",
    "twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one",
    "seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six",
    "thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine",
    "eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven",
    "twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one",
    "twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight",
    "forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five",
    "twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one",
    "nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight",
    "twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven",
    "fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four",
    "four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three",
    "forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four",
    "twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six",
    "forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one",
    "thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine",
    "twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five",
    "nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight",
    "sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five",
    "nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight",
    "thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two",
    "twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four",
    "fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine",
    "five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four",
    "thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight",
    "twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy",
    "fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine",
    "twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two",
    "twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight",
    "thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one",
    "sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five",
    "twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five",
    "twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy",
    "thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one",
    "five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four",
    "twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two",
    "nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight",
    "nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight",
    "two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one",
    "ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine",
    "forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety",
    "twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four",
    "fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine one hundred",
    "sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five",
    "sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five",
    "forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four",
    "thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty",
    "forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven",
    "fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four",
    "two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one",
    "forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine",
    "three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two",
    "twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven",
    "forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three",
    "eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven",
    "twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine",
    "twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy",
    "five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four",
    "twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven",
    "one two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty",
    "seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six",
    "forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three",
    "forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five",
    "fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four",
    "forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven",
    "thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five",
    "sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five",
    "one two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty",
    "thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven",
    "thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four",
    "twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven",
    "forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six",
    "nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight",
    "eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty",
    "twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four",
    "nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight",
    "thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two",
    "thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three",
    "thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty",
    "thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine",
    "three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two",
    "forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety",
    "forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight",
    "fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four",
    "thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty",
    "thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three",
    "nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight",
    "thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six",
    "nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight",
    "forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven",
    "forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two",
    "forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four",
    "forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety",
    "nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight",
    "twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine",
    "forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six",
    "thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two",
    "eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty",
    "twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one",
    "forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine",
    "nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight",
    "fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four",
    "twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six",
    "fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four",
    "thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two",
    "seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six",
    "forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight",
    "thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two",
    "forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three",
    "forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four",
    "twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine",
    "forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four",
    "forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five",
    "fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine",
    "two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one",
    "forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four",
    "forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six",
    "forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven",
    "eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven",
    "four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three",
    "twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two",
    "forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six",
    "twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three",
    "fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four",
    "twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy",
    "six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five",
    "eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty",
    "forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five",
    "fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three",
    "forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one",
    "forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine",
    "thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two",
    "six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five",
    "six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five",
    "thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four",
    "thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three",
    "fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three",
    "fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine one hundred",
    "fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine",
    "twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three",
    "twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five",
    "nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight",
    "eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty",
    "twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six",
    "fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine",
    "twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four",
    "thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight",
    "twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two",
    "eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven",
    "twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine",
    "thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three",
    "seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six",
    "thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two",
    "fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine one hundred",
    "twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six",
    "eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty",
    "thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six",
    "thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two",
    "twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine",
    "thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two",
    "thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two",
    "four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three",
    "nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight",
    "thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one",
    "seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six",
    "forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four",
    "forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six",
    "twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one",
    "thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two",
    "two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one",
    "twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine",
    "two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one",
    "thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven",
    "forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six",
    "thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two",
    "fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three",
    "thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four",
    "twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three",
    "forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one",
    "twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine",
    "three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two",
    "one two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty",
    "eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven",
    "thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven",
    "ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine",
    "eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty",
    "seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six",
    "three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two",
    "fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine",
    "forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight",
    "forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one",
    "seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six",
    "forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two",
    "twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one",
    "forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one",
    "thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine",
    "six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five",
    "nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight",
    "thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four",
    "eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty",
    "thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three",
    "forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine",
    "forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five",
    "thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine",
    "forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety",
    "forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five",
    "thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine",
    "eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven",
    "forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine",
    "thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two",
    "eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven",
    "four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three",
    "twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one",
    "three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two",
    "forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight",
    "fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four",
    "thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty",
    "thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven",
    "forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety",
    "twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven",
    "thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine",
    "thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven",
    "twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six",
    "two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one",
    "thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight",
    "forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four",
    "twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six",
    "forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five",
    "twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven",
    "thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight",
    "seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six",
    "twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one",
    "eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty",
    "seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six",
    "twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three",
    "twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five",
    "nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight",
    "six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five",
    "thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two",
    "thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four",
    "forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six",
    "ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine",
    "thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight",
    "forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety",
    "fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four",
    "forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three",
    "twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five",
    "thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two",
    "forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine",
    "twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine",
    "three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two",
    "twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three",
    "nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight",
    "thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four",
    "forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety",
    "seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six",
    "two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one",
    "thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty",
    "twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five",
    "twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy",
    "eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven",
    "forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three",
    "seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six",
    "thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight",
    "thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five",
    "forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety",
    "twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one",
    "eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty",
    "thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six",
    "thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty",
    "forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two",
    "four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three",
    "seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six",
    "forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five",
    "ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine",
    "forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven",
    "forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three",
    "fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three",
    "twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six",
    "eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven",
    "seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six",
    "forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three",
    "fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine",
    "eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty",
    "twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four",
    "twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight",
    "forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two",
    "nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight",
    "fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three",
    "forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine",
    "twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six",
    "twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one",
    "thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two",
    "ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine",
    "forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven",
    "twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven",
    "forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety",
    "fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine one hundred",
    "twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five",
    "seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six",
    "four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three",
    "fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three",
    "forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two",
    "nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight",
    "forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven",
    "seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six",
    "twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy",
    "nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight",
    "thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five",
    "fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four",
    "forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety",
    "fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine one hundred",
    "thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight",
    "fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four",
    "twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four",
    "twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one",
    "seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six",
    "twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one",
    "thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven",
    "twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy",
    "forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine",
    "five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four",
    "seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six",
    "thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine",
    "sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five",
    "twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one",
    "forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three",
    "fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine",
    "twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five",
    "twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one",
    "thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven",
    "fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four",
    "forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight",
    "one two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty",
    "thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight",
    "two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one",
    "two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one",
    "twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four",
    "seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six",
    "forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety",
    "twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one",
    "thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six",
    "forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four",
    "eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty",
    "twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four",
    "eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven",
    "twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three",
    "fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine",
    "twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three",
    "eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven",
    "thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two",
    "five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four",
    "twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one",
    "twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven",
    "three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two",
    "thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two",
    "thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one",
    "forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight",
    "thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three",
    "forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven",
    "forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four",
    "twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three",
    "twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine",
    "twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight",
    "twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine",
    "forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine",
    "twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five",
    "fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four",
    "forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five",
    "forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven",
    "one two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty",
    "seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six",
    "twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three",
    "forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two",
    "thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight",
    "thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two",
    "thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty",
    "thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three",
    "five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four",
    "thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four",
    "fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine one hundred",
    "forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one",
    "four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three",
    "twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five",
    "thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two",
    "forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine",
    "twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four",
    "fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three",
    "seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six",
    "eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven",
    "eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven",
    "forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six",
    "twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five",
    "three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two",
    "six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five",
    "forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight",
    "nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight",
    "four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three",
    "thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven",
    "thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight",
    "thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six",
    "four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three",
    "twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two",
    "twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy",
    "twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one",
    "thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight",
    "thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one",
    "fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine one hundred",
    "twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six",
    "twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six",
    "thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four",
    "five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four",
    "twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy",
    "four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three",
    "fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three",
    "forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four",
    "thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight",
    "seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six",
    "twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three",
    "forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three",
    "twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six",
    "twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one",
    "four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three",
    "ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine",
    "thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two",
    "thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five",
    "forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two",
    "twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one",
    "forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two",
    "twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight",
    "forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two",
    "forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six",
    "twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five",
    "six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five",
    "thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine",
    "thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine",
    "thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one",
    "forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four",
    "forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven",
    "thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six",
    "twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven",
    "thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two",
    "forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three",
    "ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine",
    "twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two",
    "forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three",
    "twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six",
    "fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine",
    "five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four",
    "five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four",
    "twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine",
    "one two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty",
    "ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine",
    "thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four",
    "thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight",
    "twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy",
    "thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight",
    "thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight",
    "forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three",
    "one two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty",
    "twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six",
    "seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six",
    "ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine",
    "six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five",
    "twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one",
    "twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four",
    "eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty",
    "four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three",
    "seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six",
    "six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five",
    "twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two",
    "forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four",
    "thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three",
    "twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven",
    "twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six",
    "three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two",
    "thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one",
    "thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five",
    "twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy",
    "thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine",
    "twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven",
    "twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five",
    "forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six",
    "forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight",
    "three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two",
    "twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two",
    "one two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty",
    "forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one",
    "twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five",
    "twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one",
    "eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven",
    "thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three",
    "thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three",
    "one two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty",
    "eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven",
    "twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one",
    "eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty",
    "forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one",
    "fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three",
    "twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three",
    "forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine",
    "thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five",
    "thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two",
    "forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two",
    "six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five",
    "forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five",
    "twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two",
    "thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four",
    "forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one",
    "thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one",
    "forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two",
    "thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two",
    "thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five",
    "thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine",
    "twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three",
    "two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one",
    "thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five",
    "twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one",
    "four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three",
    "six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five",
    "forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four",
    "twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine",
    "forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six",
    "twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two",
    "fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four",
    "twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four",
    "twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one",
    "forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three",
    "forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine",
    "fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three",
    "thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty",
    "thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine",
    "twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three",
    "fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three",
    "twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine",
    "thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight",
    "thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one",
    "thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two",
    "twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one",
    "forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety",
    "twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine",
    "thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six",
    "forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one",
    "thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two",
    "fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine",
    "twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three",
    "thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two",
    "thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four",
    "forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two",
    "twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three",
    "two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one",
    "forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven",
    "thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five",
    "forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight",
    "seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six",
    "forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven",
    "nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight",
    "fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine one hundred",
    "thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three",
    "forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine",
    "twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven",
    "thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven",
    "twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five",
    "thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two",
    "twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one",
    "eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven",
    "thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two",
    "forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two",
    "forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine",
    "twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven",
    "twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two",
    "forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one",
    "twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five",
    "nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight",
    "twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six",
    "nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight",
    "ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine",
    "fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine",
    "one two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty",
    "twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight",
    "forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three",
    "thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six",
    "four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three",
    "forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine",
    "thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven",
    "thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four",
    "two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one",
    "thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine",
    "forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three",
    "twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three",
    "thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four",
    "thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six",
    "thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four",
    "five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four",
    "forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three",
    "forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight",
    "six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five",
    "twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine",
    "thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty",
    "seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six",
    "four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three",
    "forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven",
    "ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine",
    "twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one",
    "seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six",
    "forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five",
    "twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine",
    "twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy",
    "one two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty",
    "thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four",
    "fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine",
    "forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight",
    "two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one",
    "thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven",
    "sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five",
    "eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven",
    "forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven",
    "thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six",
    "five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four",
    "twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine",
    "seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six",
    "six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five",
    "five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four",
    "thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three",
    "twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy",
    "nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight",
    "twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine",
    "forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine",
    "seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six",
    "eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven",
    "five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four",
    "twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five",
    "forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two",
    "forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one",
    "forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight",
    "six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five",
    "twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven",
    "twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two",
    "forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two",
    "thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven",
    "fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine",
    "thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty",
    "fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine one hundred",
    "seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six",
    "twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight",
    "forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one",
    "fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four",
    "forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five",
    "fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four",
    "seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six",
    "thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine",
    "twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three",
    "fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine",
    "thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty",
    "twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two",
    "thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two",
    "twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine",
    "twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one",
    "fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine one hundred",
    "twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one",
    "five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four",
    "six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five",
    "thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three",
    "twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four",
    "forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven",
    "one two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty",
    "thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine",
    "forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five",
    "thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one",
    "nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight",
    "thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one",
    "four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three",
    "twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven",
    "six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five",
    "seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six",
    "two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one",
    "fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four",
    "twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six",
    "eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven",
    "thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight",
    "twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six",
    "seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six",
    "thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six",
    "nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight",
    "forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one",
    "seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six",
    "thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three",
    "forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine",
    "six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five",
    "three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two",
    "ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine",
    "fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine",
    "fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine",
    "thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one",
    "twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine",
    "twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six",
    "twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two",
    "twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three",
    "forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four",
    "thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty",
    "ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine",
    "forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine",
    "forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five",
    "twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four",
    "forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six",
    "thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four",
    "nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight",
    "six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five",
    "nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight",
    "thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven",
    "twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one",
    "two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one",
    "sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five",
    "nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight",
    "twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight",
    "thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine",
    "thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty",
    "twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight",
    "eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven",
    "twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three",
    "two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one",
    "forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four",
    "thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one",
    "four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three",
    "four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three",
    "thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five",
    "thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven",
    "twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three",
    "fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three",
    "twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one",
    "forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two",
    "twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight",
    "twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four",
    "four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three",
    "eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty",
    "thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five",
    "twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four",
    "five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four",
    "eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty",
    "two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one",
    "twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five",
    "forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight",
    "nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight",
    "forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety",
    "thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight",
    "fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four",
    "twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy",
    "thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four",
    "thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five",
    "five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four",
    "thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three",
    "twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four",
    "twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five",
    "twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one",
    "eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven",
    "twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three",
    "nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight",
    "fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three",
    "twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven",
    "thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one",
    "forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine",
    "one two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty",
    "thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five",
    "nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight",
    "thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty",
    "thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five",
    "forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five",
    "seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six",
    "thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one",
    "two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one",
    "forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two",
    "fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine one hundred",
    "thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two",
    "thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two",
    "eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven",
    "twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one",
    "forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one",
    "twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven",
    "thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two",
    "thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two",
    "forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one",
    "forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight",
    "forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three",
    "eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty",
    "twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five",
    "twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one",
    "forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one",
    "forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven",
    "thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three",
    "seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six",
    "thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four",
    "thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five",
    "fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine one hundred",
    "thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine",
    "forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three",
    "twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six",
    "thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three",
    "twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine",
    "twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five",
    "forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six",
    "twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven",
    "thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight",
    "nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight",
    "forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight",
    "twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight",
    "thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five",
    "eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty",
    "seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six",
    "twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one",
    "twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three",
    "twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two",
    "twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one",
    "nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight",
    "eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven",
    "thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight",
    "twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight",
    "twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy",
    "forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one",
    "thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three",
    "twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one",
    "eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty",
    "eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty",
    "thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six",
    "twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two",
    "thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four",
    "eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty",
    "four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three",
    "thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two",
    "eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven",
    "twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one",
    "twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine",
    "thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five",
    "thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one",
    "twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven",
    "eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty",
    "two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one",
    "twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven",
    "nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight",
    "fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine",
    "thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one",
    "sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five",
    "forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three",
    "forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three",
    "twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one",
    "thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine",
    "thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one",
    "twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five",
    "five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four",
    "eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven",
    "forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five",
    "twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy",
    "fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine",
    "twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two",
    "thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five",
    "forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine",
    "fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine one hundred",
    "twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five",
    "twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight",
    "thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight",
    "three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two",
    "thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two",
    "thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four",
    "thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine",
    "thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight",
    "thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five",
    "thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven",
    "four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three",
    "thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five",
    "seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six",
    "twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight",
    "thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six",
    "eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven",
    "eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven",
    "fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four",
    "four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three",
    "six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five",
    "twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight",
    "fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three",
    "forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three",
    "thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight",
    "forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six",
    "twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four",
    "thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five",
    "fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine",
    "ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine",
    "forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six",
    "twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one",
    "forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one",
    "twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine",
    "thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one",
    "fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four",
    "twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two",
    "forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six",
    "twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two",
    "seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six",
    "forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven",
    "five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four",
    "twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two",
    "thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four",
    "forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine",
    "sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five",
    "ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine",
    "ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine",
    "forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight",
    "six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five",
    "forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three",
    "thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight",
    "fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four",
    "seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six",
    "forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six",
    "twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four",
    "forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six",
    "twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven",
    "thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty",
    "thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two",
    "forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety",
    "thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one",
    "twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five",
    "forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six",
    "sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five",
    "forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four",
    "twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven",
    "fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine one hundred",
    "fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four",
    "eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven",
    "twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one",
    "ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine",
    "fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four",
    "eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven",
    "thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three",
    "ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine",
    "fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine one hundred",
    "fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine one hundred",
    "thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four",
    "one two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty",
    "forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine",
    "thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty",
    "eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven",
    "nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight",
    "sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five",
    "twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five",
    "sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five",
    "nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight",
    "forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three",
    "three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two",
    "one two three four five six seven eight nine ten eleven twelve thirteen fourteen fifteen sixteen seventeen eighteen nineteen twenty twenty one twenty two twenty three twenty four twenty five twenty six twenty seven twenty eight twenty nine thirty thirty one thirty two thirty three thirty four thirty five thirty six thirty seven thirty eight thirty nine forty forty one forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty",
    "forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five",
    "fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one ninety two ninety three ninety four ninety five ninety six ninety seven ninety eight ninety nine"
]

print(len(texts))

1000


In [4]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

# Train a BPE tokenizer
tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
tokenizer.pre_tokenizer = Whitespace()
trainer = BpeTrainer(vocab_size=500, special_tokens=["[UNK]", "[PAD]", "[EOS]"])
tokenizer.train_from_iterator(texts, trainer)

# Save and load the tokenizer
tokenizer.save("bpe_tokenizer.json")
tokenizer = Tokenizer.from_file("bpe_tokenizer.json")

In [5]:
import torch.nn as nn

class RMSNorm(nn.Module):
    def __init__(self, dim, eps=1e-5):
        super().__init__()
        self.scale = torch.nn.Parameter(torch.ones(dim))
        self.eps = eps

    def forward(self, x):
        norm = torch.norm(x, dim=-1, keepdim=True)
        return x / (norm + self.eps) * self.scale

In [6]:
import torch
import torch.nn as nn

class GeGLU(nn.Module):
    def __init__(self, dim_in, dim_out):
        super(GeGLU, self).__init__()
        self.proj = nn.Linear(dim_in, dim_out * 2)

    def forward(self, x):
        x, gate = self.proj(x).chunk(2, dim=-1)
        return x * torch.sigmoid(gate)

In [7]:
import torch.nn.functional as F

class GroupedAttentionLayer(nn.Module):
    def __init__(self, dim, num_heads, num_groups, max_seq_len, device='cpu'):
        super(GroupedAttentionLayer, self).__init__()
        self.dim = dim
        self.num_heads = num_heads
        self.num_groups = num_groups
        self.head_dim = dim // num_heads
        self.max_seq_len = max_seq_len
        self.device = device

        # Linear projections for Q, K, V
        self.q_linear = nn.Linear(dim, dim)
        self.k_linear = nn.Linear(dim, dim)
        self.v_linear = nn.Linear(dim, dim)
        self.out_linear = nn.Linear(dim, dim)

    def forward(self, x):
        batch_size, seq_len, dim = x.shape

        # Precompute the RoPE embeddings
        rope_sin, rope_cos = create_1d_rope_embeddings(seq_len, self.head_dim, self.device)

        # Linear projections
        q = self.q_linear(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        k = self.k_linear(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)
        v = self.v_linear(x).view(batch_size, seq_len, self.num_heads, self.head_dim).transpose(1, 2)

        # Apply RoPE to Q, K, V
        q = apply_1d_rope(q, rope_sin, rope_cos)
        k = apply_1d_rope(k, rope_sin, rope_cos)
        v = apply_1d_rope(v, rope_sin, rope_cos)

        # Group the queries, keys, and values
        q = q.view(batch_size, self.num_heads, self.num_groups, seq_len, self.head_dim // self.num_groups)
        k = k.view(batch_size, self.num_heads, self.num_groups, seq_len, self.head_dim // self.num_groups)
        v = v.view(batch_size, self.num_heads, self.num_groups, seq_len, self.head_dim // self.num_groups)

        # Compute attention using scaled_dot_product_attention
        attn_output = []
        for group in range(self.num_groups):
            attn_output.append(F.scaled_dot_product_attention(
                q[:, :, group], k[:, :, group], v[:, :, group], attn_mask=None, dropout_p=0.0, is_causal=True
            ))
        attn_output = torch.cat(attn_output, dim=2)

        # Reshape the output
        out = attn_output.transpose(1, 2).contiguous().view(batch_size, seq_len, dim)

        # Linear projection
        out = self.out_linear(out)

        return out

class TransformerLayer(nn.Module):
    def __init__(self, dim, num_heads, num_groups, max_seq_len, device='cpu'):
        super(TransformerLayer, self).__init__()
        self.attention = GroupedAttentionLayer(dim, num_heads, num_groups, max_seq_len, device)
        self.norm1 = RMSNorm(dim)
        self.norm2 = RMSNorm(dim)
        self.ffn = nn.Sequential(
            nn.Linear(dim, dim * 4),
            GeGLU(dim * 4, dim * 4),
            nn.Linear(dim * 4, dim)
        )

    def forward(self, x):
        # Apply attention and add & norm
        x = self.norm1(x + self.attention(x))
        # Apply feed-forward network and add & norm
        x = self.norm2(x + self.ffn(x))
        return x

class TransformerModel(nn.Module):
    def __init__(self, num_layers, dim, num_heads, num_groups, max_seq_len, vocab_size, device='cpu'):
        super(TransformerModel, self).__init__()
        self.token_embedding = nn.Embedding(vocab_size, dim)
        self.layers = nn.ModuleList([
            TransformerLayer(dim, num_heads, num_groups, max_seq_len, device)
            for _ in range(num_layers)
        ])
        self.ln_f = RMSNorm(dim)
        self.head = nn.Linear(dim, vocab_size)

    def forward(self, x):
        # Token embeddings
        x = self.token_embedding(x)
        # Apply transformer layers
        for layer in self.layers:
            x = layer(x)
        # Final layer norm
        x = self.ln_f(x)
        # Output logits
        logits = self.head(x)
        return logits

In [8]:
import torch

# Tokenize the texts and ensure tensors are of type torch.long
encoded = tokenizer.encode_batch(texts)
sequences = [torch.tensor(enc.ids, dtype=torch.long) for enc in encoded]

# Pad sequences to the same length with explicit dtype
pad_token_id = tokenizer.token_to_id("[PAD]")
seq_len = max(len(seq) for seq in sequences)
sequences_padded = [torch.cat([seq, torch.tensor([pad_token_id] * (seq_len - len(seq)), dtype=torch.long)]) for seq in sequences]

# Create input and target tensors with dtype=torch.long
inputs = torch.stack(sequences_padded)
targets = torch.stack([torch.cat([seq[1:], torch.tensor([pad_token_id], dtype=torch.long)]) for seq in sequences_padded])

dataset = torch.utils.data.TensorDataset(inputs, targets)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=512, shuffle=True)

In [9]:
print(tokenizer.decode(inputs[5].squeeze().tolist(), skip_special_tokens=False))
print(tokenizer.decode(targets[5].squeeze().tolist(), skip_special_tokens=False))

forty two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety ninety one [PAD]
two forty three forty four forty five forty six forty seven forty eight forty nine fifty fifty one fifty two fifty three fifty four fifty five fifty six fifty seven fifty eight fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one e

In [10]:
from pytorch_optimizer import get_supported_optimizers

supported_optimizers = get_supported_optimizers()

print(supported_optimizers)

['a2grad', 'accsgd', 'adabelief', 'adabound', 'adadelta', 'adafactor', 'adahessian', 'adai', 'adalite', 'adalomo', 'adamax', 'adamg', 'adammini', 'adamod', 'adamp', 'adams', 'adamw', 'adan', 'adanorm', 'adapnm', 'adashift', 'adasmooth', 'ademamix', 'aggmo', 'aida', 'alig', 'amos', 'apollo', 'asgd', 'avagrad', 'bsam', 'came', 'dadaptadagrad', 'dadaptadam', 'dadaptadan', 'dadaptlion', 'dadaptsgd', 'diffgrad', 'fadam', 'fromage', 'galore', 'gravity', 'grokfastadamw', 'kate', 'lamb', 'lars', 'lion', 'lomo', 'madgrad', 'msvag', 'nero', 'novograd', 'padam', 'pid', 'pnm', 'prodigy', 'qhadam', 'qhm', 'radam', 'ranger', 'ranger21', 'scalableshampoo', 'schedulefreeadamw', 'schedulefreesgd', 'sgdp', 'sgdw', 'shampoo', 'signsgd', 'sm3', 'soap', 'sophiah', 'srmm', 'stableadamw', 'swats', 'tiger', 'yogi']


In [11]:
from pytorch_optimizer import create_optimizer

# Training Loop
model = TransformerModel(num_layers=4, dim=128, num_heads=8, num_groups=4, max_seq_len=seq_len, vocab_size=tokenizer.get_vocab_size(), device='cuda')
model.to('cuda')

optimizer = create_optimizer(
    model,
    'soap',
    lr=1e-3,
    weight_decay=0.01,
    betas=(.95, .95),
    precondition_frequency=10
)

loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer.token_to_id("[PAD]"))

# In the training loop, ensure inputs and targets are on the correct device
for epoch in range(2000):
    total_loss = 0
    for batch in dataloader:
        input_batch, target_batch = batch
        input_batch = input_batch.to('cuda', dtype=torch.long)
        target_batch = target_batch.to('cuda', dtype=torch.long)
        logits = model(input_batch)
        loss = loss_fn(logits.view(-1, tokenizer.get_vocab_size()), target_batch.view(-1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss / len(dataloader)}")

Epoch 1, Loss: 4.35879921913147
Epoch 2, Loss: 4.344306945800781
Epoch 3, Loss: 4.316580772399902
Epoch 4, Loss: 4.2728540897369385
Epoch 5, Loss: 4.2255589962005615
Epoch 6, Loss: 4.207679271697998
Epoch 7, Loss: 4.2009618282318115
Epoch 8, Loss: 4.193957090377808
Epoch 9, Loss: 4.186695337295532
Epoch 10, Loss: 4.179203748703003
Epoch 11, Loss: 4.172108173370361
Epoch 12, Loss: 4.166296482086182
Epoch 13, Loss: 4.160719156265259
Epoch 14, Loss: 4.1551878452301025
Epoch 15, Loss: 4.149492263793945
Epoch 16, Loss: 4.143577337265015
Epoch 17, Loss: 4.137955665588379
Epoch 18, Loss: 4.131902694702148
Epoch 19, Loss: 4.125159740447998
Epoch 20, Loss: 4.1174256801605225
Epoch 21, Loss: 4.108832836151123
Epoch 22, Loss: 4.100235223770142
Epoch 23, Loss: 4.090587377548218
Epoch 24, Loss: 4.079275608062744
Epoch 25, Loss: 4.066107988357544
Epoch 26, Loss: 4.0510218143463135
Epoch 27, Loss: 4.036371469497681
Epoch 28, Loss: 4.020110607147217
Epoch 29, Loss: 4.00155770778656
Epoch 30, Loss: 3.9

In [13]:
def infer(model, tokenizer, input_text, max_length=50, device='cuda'):
    # Tokenize the input text
    encoded = tokenizer.encode(input_text)
    input_ids = torch.tensor(encoded.ids, dtype=torch.long).unsqueeze(0).to(device)

    # Generate text
    generated_ids = input_ids
    with torch.no_grad():
        for _ in range(max_length):
            # Pass the input through the model
            outputs = model(generated_ids)
            # Get the last token logits
            next_token_logits = outputs[:, -1, :]
            # Sample the next token
            next_token_id = torch.argmax(next_token_logits, dim=-1).unsqueeze(0)
            # Append the next token to the sequence
            generated_ids = torch.cat([generated_ids, next_token_id], dim=-1)
            # Stop if the EOS token is generated
            if next_token_id.item() == tokenizer.token_to_id("[EOS]"):
                break

    # Decode the generated sequence
    decoded_text = tokenizer.decode(generated_ids.squeeze().tolist(), skip_special_tokens=True)
    return decoded_text

# Example usage
input_text = "fifty nine sixty sixty one sixty two sixty three sixty four"
generated_text = infer(model, tokenizer, input_text, max_length=50)
print(f"Generated text: {generated_text}")

Generated text: fifty nine sixty sixty one sixty two sixty three sixty four sixty five sixty six sixty seven sixty eight sixty nine seventy seventy seventy one seventy two seventy three seventy four seventy five seventy six seventy seven seventy eight seventy nine eighty eighty one eighty two eighty three eighty four eighty five eighty six eighty seven eighty eight eighty nine ninety
